**Tugas Scheduling Algorithm**

Kelompok 1 / IMT 23:

Alicia Juanita Lisal (0806022310002)

Andrey Hartawan Suwardi (0806022310021)

Excel Marcello Parinussa (0806022310029)

Felicia Wijaya (0806022310007)

In [11]:
import pandas as pd
from collections import deque
import copy

def load_processes_from_excel(file_path):
    df = pd.read_excel(file_path)
    processes = []
    for _, row in df.iterrows():
        processes.append({
            "pid": row['PID'],
            "arrival_time": row['arrival_time'],
            "burst_time": row['Burst_time']
        })
    return processes

Fungsi-fungsi Algoritma Scheduling

In [10]:
class ProcessScheduler:
    def __init__(self, processes):
        self.processes = processes
        self.timeline = []
        self.process_summary = []

    def create_timeline_df(self):
        df = pd.DataFrame(self.timeline)
        df.columns = ['Number', 'Second', 'PID']
        return df

    def create_summary_df(self):
        df = pd.DataFrame(self.process_summary)
        return df

    def calculate_averages(self):
        if not self.process_summary:
            return None

        total_waiting_time = sum(p['waiting_time'] for p in self.process_summary)
        total_turnaround_time = sum(p['complete_time'] - p['arrival_time'] for p in self.process_summary)
        n_processes = len(self.process_summary)

        return {
            'average_waiting_time': total_waiting_time / n_processes,
            'average_turnaround_time': total_turnaround_time / n_processes
        }

    def print_process_summary(self):
        for process in self.process_summary:
            print(f"process : {process['pid']}, arrival_time : {process['arrival_time']}, " +
                  f"burst_time : {process['burst_time']}, complete_time : {process['complete_time']}, " +
                  f"waiting_time : {process['waiting_time']}")

        averages = self.calculate_averages()
        if averages:
            print(f"\nAverages:")
            print(f"Average Waiting Time: {averages['average_waiting_time']:.2f}")
            print(f"Average Turnaround Time: {averages['average_turnaround_time']:.2f}")

    def fcfs(self):
        self.timeline = []
        self.process_summary = []
        sorted_processes = sorted(self.processes, key=lambda x: x["arrival_time"])
        current_time = 0
        counter = 0

        for process in sorted_processes:
            # Handle idle time
            while current_time < process["arrival_time"]:
                self.timeline.append([counter, current_time, "IDLE"])
                current_time += 1
                counter += 1

            waiting_time = current_time - process["arrival_time"]

            # Process execution
            remaining_time = process["burst_time"]
            while remaining_time > 0:
                self.timeline.append([counter, current_time, process["pid"]])
                current_time += 1
                remaining_time -= 1
                counter += 1

            # Add to process summary
            self.process_summary.append({
                "pid": process["pid"],
                "arrival_time": process["arrival_time"],
                "burst_time": process["burst_time"],
                "complete_time": current_time,
                "waiting_time": waiting_time
            })

        return self.timeline, self.process_summary

    def sjf_non_preemptive(self):
        self.timeline = []
        self.process_summary = []
        processes = sorted(self.processes, key=lambda x: x["arrival_time"])
        current_time = 0
        counter = 0
        ready_queue = []
        completed = []
        processes_copy = processes.copy()

        while len(completed) < len(processes):
            # Add arrived processes to ready queue
            while processes_copy and processes_copy[0]["arrival_time"] <= current_time:
                ready_queue.append(processes_copy.pop(0))

            if not ready_queue:
                self.timeline.append([counter, current_time, "IDLE"])
                current_time += 1
                counter += 1
                continue

            # Sort ready queue by burst time
            ready_queue.sort(key=lambda x: x["burst_time"])
            process = ready_queue.pop(0)
            waiting_time = current_time - process["arrival_time"]

            # Process execution
            remaining_time = process["burst_time"]
            while remaining_time > 0:
                self.timeline.append([counter, current_time, process["pid"]])
                current_time += 1
                remaining_time -= 1
                counter += 1

            # Add to process summary
            self.process_summary.append({
                "pid": process["pid"],
                "arrival_time": process["arrival_time"],
                "burst_time": process["burst_time"],
                "complete_time": current_time,
                "waiting_time": waiting_time
            })

            completed.append(process)

            # Add any new arrivals to ready queue
            while processes_copy and processes_copy[0]["arrival_time"] <= current_time:
                ready_queue.append(processes_copy.pop(0))

        return self.timeline, self.process_summary

    def sjf_preemptive(self):
        self.timeline = []
        self.process_summary = []
        processes = sorted(self.processes, key=lambda x: x["arrival_time"])
        current_time = 0
        counter = 0
        remaining_bt = {p["pid"]: p["burst_time"] for p in processes}
        completion_times = {}
        waiting_times = {p["pid"]: 0 for p in processes}
        last_run_time = {p["pid"]: 0 for p in processes}

        while any(remaining_bt.values()):
            # Find process with shortest remaining time
            shortest = None
            min_bt = float('inf')

            for process in processes:
                if process["arrival_time"] <= current_time and remaining_bt[process["pid"]] > 0:
                    if remaining_bt[process["pid"]] < min_bt:
                        min_bt = remaining_bt[process["pid"]]
                        shortest = process

            if shortest is None:
                self.timeline.append([counter, current_time, "IDLE"])
                current_time += 1
                counter += 1
                continue

            # Execute process for 1 time unit
            self.timeline.append([counter, current_time, shortest["pid"]])
            remaining_bt[shortest["pid"]] -= 1

            # Update waiting time
            for process in processes:
                if (process["pid"] != shortest["pid"] and
                    process["arrival_time"] <= current_time and
                    remaining_bt[process["pid"]] > 0):
                    waiting_times[process["pid"]] += 1

            current_time += 1
            counter += 1

            # Check if process is completed
            if remaining_bt[shortest["pid"]] == 0:
                completion_times[shortest["pid"]] = current_time

        # Create process summary
        for process in processes:
            self.process_summary.append({
                "pid": process["pid"],
                "arrival_time": process["arrival_time"],
                "burst_time": process["burst_time"],
                "complete_time": completion_times[process["pid"]],
                "waiting_time": waiting_times[process["pid"]]
            })

        return self.timeline, self.process_summary

    def round_robin(self, quantum=12):
        self.timeline = []
        self.process_summary = []
        processes = sorted(self.processes, key=lambda x: x["arrival_time"])
        current_time = 0
        counter = 0

        # Initialize remaining burst times and other tracking variables
        remaining_bt = {p["pid"]: p["burst_time"] for p in processes}
        completion_times = {}
        waiting_times = {p["pid"]: 0 for p in processes}
        ready_queue = deque()
        processes_copy = processes.copy()

        while any(remaining_bt.values()) or ready_queue:
            # Add newly arrived processes to ready queue
            while processes_copy and processes_copy[0]["arrival_time"] <= current_time:
                ready_queue.append(processes_copy.pop(0))

            if not ready_queue:
                self.timeline.append([counter, current_time, "IDLE"])
                current_time += 1
                counter += 1
                continue

            # Get next process from ready queue
            current_process = ready_queue.popleft()

            # Execute for quantum time or remaining burst time, whichever is smaller
            execution_time = min(quantum, remaining_bt[current_process["pid"]])

            # Update waiting times for other processes in ready queue
            for pid in remaining_bt:
                if (pid != current_process["pid"] and
                    remaining_bt[pid] > 0 and
                    any(p["pid"] == pid and p["arrival_time"] <= current_time for p in processes)):
                    waiting_times[pid] += execution_time

            # Execute process
            for _ in range(execution_time):
                self.timeline.append([counter, current_time, current_process["pid"]])
                current_time += 1
                counter += 1
                remaining_bt[current_process["pid"]] -= 1

                # Add any new arrivals to ready queue
                while processes_copy and processes_copy[0]["arrival_time"] <= current_time:
                    ready_queue.append(processes_copy.pop(0))

            # If process is not finished, add it back to ready queue
            if remaining_bt[current_process["pid"]] > 0:
                ready_queue.append(current_process)
            else:
                completion_times[current_process["pid"]] = current_time

        # Create process summary
        for process in processes:
            self.process_summary.append({
                "pid": process["pid"],
                "arrival_time": process["arrival_time"],
                "burst_time": process["burst_time"],
                "complete_time": completion_times[process["pid"]],
                "waiting_time": waiting_times[process["pid"]]
            })

        return self.timeline, self.process_summary

    def ljf_non_preemptive(self):
        self.timeline = []
        self.process_summary = []
        processes = sorted(self.processes, key=lambda x: x["arrival_time"])
        current_time = 0
        counter = 0
        ready_queue = []
        completed = []
        processes_copy = processes.copy()

        while len(completed) < len(processes):
            # Add arrived processes to ready queue
            while processes_copy and processes_copy[0]["arrival_time"] <= current_time:
                ready_queue.append(processes_copy.pop(0))

            if not ready_queue:
                self.timeline.append([counter, current_time, "IDLE"])
                current_time += 1
                counter += 1
                continue

            # Sort ready queue by burst time (descending for LJF)
            ready_queue.sort(key=lambda x: x["burst_time"], reverse=True)
            process = ready_queue.pop(0)
            waiting_time = current_time - process["arrival_time"]

            # Process execution
            remaining_time = process["burst_time"]
            while remaining_time > 0:
                self.timeline.append([counter, current_time, process["pid"]])
                current_time += 1
                remaining_time -= 1
                counter += 1

            # Add to process summary
            self.process_summary.append({
                "pid": process["pid"],
                "arrival_time": process["arrival_time"],
                "burst_time": process["burst_time"],
                "complete_time": current_time,
                "waiting_time": waiting_time
            })

            completed.append(process)

            # Add any new arrivals to ready queue
            while processes_copy and processes_copy[0]["arrival_time"] <= current_time:
                ready_queue.append(processes_copy.pop(0))

        return self.timeline, self.process_summary

    def save_results(self, output_file):
        # Create Excel writer object
        with pd.ExcelWriter(output_file) as writer:
            # Save timeline
            timeline_df = self.create_timeline_df()
            timeline_df.to_excel(writer, sheet_name='Timeline', index=False)

            # Save process summary
            summary_df = self.create_summary_df()
            summary_df.to_excel(writer, sheet_name='Process_Summary', index=False)

            # Save averages
            averages = self.calculate_averages()
            if averages:
                avg_df = pd.DataFrame([averages])
                avg_df.to_excel(writer, sheet_name='Averages', index=False)

In [12]:
def main():
    # Load processes from Excel
    input_file = 'processes.xlsx'
    processes = load_processes_from_excel(input_file)

    # Create scheduler instance
    scheduler = ProcessScheduler(processes)

    # Run different algorithms and save results
    algorithms = {
        'FCFS': scheduler.fcfs,
        'SJF_Non_Preemptive': scheduler.sjf_non_preemptive,
        'SJF_Preemptive': scheduler.sjf_preemptive,
        'Round_Robin': lambda: scheduler.round_robin(quantum=12),
        'LJF_Non_Preemptive': scheduler.ljf_non_preemptive
    }

    results_summary = []

    for algo_name, algo_func in algorithms.items():
        print(f"\n{algo_name} Results:")
        timeline, summary = algo_func()

        # Print process summary
        scheduler.print_process_summary()

        # Save to Excel
        output_file = f'{algo_name}_results.xlsx'
        scheduler.save_results(output_file)
        print(f"Results saved to {output_file}")

        # Store averages for comparison
        averages = scheduler.calculate_averages()
        if averages:
            averages['algorithm'] = algo_name
            results_summary.append(averages)

    # Create comparison DataFrame and save to Excel
    if results_summary:
        comparison_df = pd.DataFrame(results_summary)
        print("\nAlgorithm Comparison:")
        print(comparison_df)

if __name__ == "__main__":
    main()


FCFS Results:
process : P51, arrival_time : 0, burst_time : 15, complete_time : 15, waiting_time : 0
process : P66, arrival_time : 0, burst_time : 14, complete_time : 29, waiting_time : 15
process : P98, arrival_time : 0, burst_time : 14, complete_time : 43, waiting_time : 29
process : P39, arrival_time : 1, burst_time : 19, complete_time : 62, waiting_time : 42
process : P40, arrival_time : 1, burst_time : 9, complete_time : 71, waiting_time : 61
process : P90, arrival_time : 1, burst_time : 17, complete_time : 88, waiting_time : 70
process : P23, arrival_time : 2, burst_time : 16, complete_time : 104, waiting_time : 86
process : P55, arrival_time : 2, burst_time : 11, complete_time : 115, waiting_time : 102
process : P8, arrival_time : 3, burst_time : 7, complete_time : 122, waiting_time : 112
process : P21, arrival_time : 4, burst_time : 6, complete_time : 128, waiting_time : 118
process : P44, arrival_time : 4, burst_time : 17, complete_time : 145, waiting_time : 124
process : P11